## Extraction of BG from a video through median filter.
Here applied to a multiple frames. A video is then made out of it

In [3]:
# Imports
import sys, os
from math import floor, ceil
from tqdm import tqdm
from pathlib import Path
import cv2 as cv
import matplotlib.pyplot as plt
# setting path
sys.path.append('../Preprocessing')
sys.path.append('../BackgroundRemoval/6_median_filter_remove_bees')
sys.path.append('../VideoManagment')
sys.path.append('MedianFilter')
from preproc import beautify_frame
from median_image import median_filter
from videolib import generateVideoFromList

In [14]:
START_INDEX = 6890 # The index of the first image to use
END_INDEX = 7000 # The index of the last image to use
NUMB_IMAGES = 100 # Number of images used within the median filter. For a frame i, the images used are from i-floor(NUMB_IMAGES/2) to i+ceil(NUMB_IMAGES/2)
FRAME_SKIPS = 1 # Number of frames to skip in between images used for background removal. Put 1 to use all images, 2 to use every other image, etc.

img_path = '/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09-24.10_observation_OH/Images/h1r1_1minute/'

# Get the list of images
files = os.listdir(img_path)
files.sort()
files = files[START_INDEX:END_INDEX]

print("Loading images…")
images = [cv.imread(img_path+file, cv.IMREAD_GRAYSCALE) for file in tqdm(files)]

# Preprocess the image
print("Preprocessing images…")
images = [beautify_frame(img) for img in tqdm(images)]

# Median filter the video
final_imgs=[]
print("Applying median filter to all frames…")
for i in tqdm(range(len(images))):
    if i<FRAME_SKIPS*floor(NUMB_IMAGES/2) or i>=len(images)-FRAME_SKIPS*ceil(NUMB_IMAGES/2):
        continue # Not enough images around to compute the median image
    median_img = median_filter(images[i-FRAME_SKIPS*floor(NUMB_IMAGES/2):i+FRAME_SKIPS*ceil(NUMB_IMAGES/2):FRAME_SKIPS])
    # Annotate the name of the file on the top left corner
    cv.putText(median_img, files[i], (10, 70), cv.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3, cv.LINE_AA)
    final_imgs.append(median_img)

# Check the length of the video
print("Length of the video: ", len(final_imgs))
# Save the video
current_dir = Path().absolute() # Directory of current working directory
save_dir = current_dir / 'MedianFilter/Outputs'
generateVideoFromList(final_imgs, dest=save_dir, name = f'median_filter_n{NUMB_IMAGES}_s{FRAME_SKIPS}', fps= 30,grayscale=True)

Loading images…


100%|██████████| 110/110 [00:03<00:00, 35.60it/s]


Preprocessing images…


100%|██████████| 110/110 [00:09<00:00, 11.68it/s]


Applying median filter to all frames…


100%|██████████| 110/110 [00:18<00:00,  5.99it/s]


Length of the video:  10
Writing video...


100%|██████████| 10/10 [00:00<00:00, 21.72it/s]
